# Minimax for Tic Tac Toe

In this Lab, we will create a perfect player for Tic Tac Toe. Because of the simple nature of the game, there's not much work you need to do to brute-force a perfect player that'll never make a mistake. It can either win or draw - it is guaranteed to never lose.

Let's start by importing the relevant libraries - in this case, you're only going to import Foundation. Foundation is essentially an "extension to the Swift Standard Library" - it includes functionality like advanced String handling, networking, and more. To quote [Apple](https://github.com/apple/swift-corelibs-foundation):

> The Foundation framework defines a base layer of functionality that is required for almost all applications. It provides primitive classes and introduces several paradigms that define functionality not provided by either the Objective-C runtime and language or Swift standard library and language.

In [ ]:
import Foundation

Next, we need to make sure that we have a game to play before we can create the AI. I won't explain this part in very much depth, because this course - again - is on building AI, not games. To begin, create the Tic Tac Toe structure:

In [ ]:
struct TicTacToe {
    var board: [[Int]] // The Tic Tac Toe board (row-major format, 3x3 array of Integers)
}

To begin with, the structure contains nothing but an uninitialized two-dimensional array of integers. This board represents the tic tac toe game in row-major form. For example, the following board:


```
 +-+-+-+
 |x| |o|
 +-+-+-+
 |o|o| |
 +-+-+-+
 |x|x|x|
 +-+-+-+
```

Is represented programmatically as:

```
[[ 1,  0, -1],
 [-1, -1,  0],
 [ 1,  1,  1]]
```

One things to note for Python programmers: what you created, a `struct`,  is similar to a `class` - but there're a few differences:

1. An initializer for the struct was automatically created with signature `init(board: [[Int]])`
1. A class is a reference type, whereas a struct is a value type. This means that, for example, when you pass classes around in Swift or Python code, you're passing a reference to a class. When you pass a struct around, you're passing around copies of that struct.

Swift also has an incredible system known as COW🐮 (copy-on-write) (yes, occasionally with the emoji). Essentially, say you write the following code:

```
struct Point {
    var x: Double
    var y: Double
}

func moveUp(location: Point) -> Point {
    var p2 = location
    p2.y -= 1
    return p2
}

let a = Point(x: 6, y: 9)
print(a)
print(moveUp(location: a))
```

Technically, a structure is a value type - therefore, Swift must pass a copy to the function for it to modify, right? Wrong! Swift is actually passing to the function a reference to the structure. However, if the function attempts to modify that reference, only then will Swift create a copy of the struct. It's very intelligent behaviour.

To find out more about value types (structs) vs. reference types (classes), [click here](https://developer.apple.com/swift/blog/?id=10).

Back to Tic Tac Toe - let's extend the Tic Tac Toe structure before to add a custom initializer that can create a 3x3 array of all zeros.

In [ ]:
extension TicTacToe {
    init() {
        self.init(board: [[Int]](repeating: [Int](repeating: 0, count: 3), count: 3)) // 3x3 array of zeros
    }
}

Next, let's implement the `isWin(for player: Int) -> Bool` function, which will check if a certain player has won, and return `true` if they have, `false` if they're lost or the game is a draw.

In [ ]:
extension TicTacToe {
    func isWin(for player: Int) -> Bool {
        let wanting = [player, player, player] // Expecting 3 in a row
        for i in board {
            if i == wanting {
                return true // Row is equal to what was expected
            }
        }
        for col in 0..<3 {
            let row = [board[0][col], board[1][col], board[2][col]]
            if row == wanting {
                return true // Column is equal to what was expected
            }
        }
        let diag1 = [board[0][0], board[1][1], board[2][2]]
        if diag1 == wanting {
            return true // Left-to-right diagonal is equal to what was expected
        }
        let diag2 = [board[0][2], board[1][1], board[2][0]]
        if diag2 == wanting {
            return true // Right-to-left diagonal is equal to what was expected
        }
        return false // No win yet for this player
    }
}

Let's also implement the following simple computed properties for our convenience:

1. `legalMoves` - an array of `(x, y)` coordinates that represent tiles on the board that are still empty.
1. `hasWinner` - a boolean which is `true` if either player has won, `false` if the game is a draw or is in progress.
1. `isOver` - a boolean which is `true` is either player has won or the game is draw, `false` if the game is in progress.

In [ ]:
extension TicTacToe {
    var legalMoves: [(Int, Int)] {
        var legals: [(Int, Int)] = []
        for row in 0..<3 {
            for col in 0..<3 {
                if board[row][col] == 0 {
                    legals.append((row, col))
                }
            }
        }
        return legals
    }
    
    var hasWinner: Bool {
        return isWin(for: 1) || isWin(for: -1)
    }
    
    var isOver: Bool {
        return hasWinner || legalMoves.isEmpty
    }
}

Let's also implement a function that enables a player to place their symbol on a tile:

In [ ]:
extension TicTacToe {
    mutating func play(at location: (Int, Int), player: Int) {
        guard board[location.0][location.1] == 0 else {
            fatalError()
        }
        board[location.0][location.1] = player
    }
}

Now, let's implement a function that can return the "children" of a board state. The children are all possible next board states for a certain player's turn.

In [ ]:
extension TicTacToe {
    func children(player: Int) -> [TicTacToe] {
        var nextMoves: [TicTacToe] = []
        for i in legalMoves {
            var copy = TicTacToe(board: board)
            copy.play(at: i, player: player)
            nextMoves.append(copy)
        }
        return nextMoves
    }
}

Now, let's implement a simple function for convenience - the `winner` function. This will basically act as a wrapper around the `isWin` function.

Essentially, it'll return a positive value if player 1 won, and a negative value if player 2 (represented as -1) won. It'll return 0 if it's a draw or the game is in progress.

The way the return value for a win is determined is based off of the `depth` - and this comes from the Minimax algorithm. Essentially, when the Minimax algorithm is looking really deep inside the game tree, and sees a win/loss many moves ahead, it needs to be penalized based off of how deep the game tree is. This makes it so that the minimax also takes into account the "straightforwardness" of certain moves.

So, by taking a positive 10 value and subtracting the depth, or by taking the depth and subtracting a 10, you get your player 1 and player 2 scores respectively.

In [ ]:
extension TicTacToe {
    func winner(depth: Int) -> Int {
        if isWin(for: 1) {
            return 10 - depth
        }
        if isWin(for: -1) {
            return depth - 10
        }
        return 0
    }
}

Finally, it's time to make things pretty with some pretty-printing functions:

In [ ]:
extension TicTacToe {
    // Turn a number into a character
    //  0 = " "
    // +1 = "x"
    // -1 = "o"
    static func character(_ x: Int) -> String {
        x == 0 ? " " : (x == 1 ? "x" : "o")
    }
    
    // Print the board
    func display() {
        var boardTemplate = """
                               +-+-+-+
                               |1|2|3|
                               +-+-+-+
                               |4|5|6|
                               +-+-+-+
                               |7|8|9|
                               +-+-+-+
                            """
        boardTemplate = boardTemplate.replacingOccurrences(of: "1", with: "\(TicTacToe.character(board[0][0]))")
        boardTemplate = boardTemplate.replacingOccurrences(of: "2", with: "\(TicTacToe.character(board[0][1]))")
        boardTemplate = boardTemplate.replacingOccurrences(of: "3", with: "\(TicTacToe.character(board[0][2]))")
        boardTemplate = boardTemplate.replacingOccurrences(of: "4", with: "\(TicTacToe.character(board[1][0]))")
        boardTemplate = boardTemplate.replacingOccurrences(of: "5", with: "\(TicTacToe.character(board[1][1]))")
        boardTemplate = boardTemplate.replacingOccurrences(of: "6", with: "\(TicTacToe.character(board[1][2]))")
        boardTemplate = boardTemplate.replacingOccurrences(of: "7", with: "\(TicTacToe.character(board[2][0]))")
        boardTemplate = boardTemplate.replacingOccurrences(of: "8", with: "\(TicTacToe.character(board[2][2]))")
        boardTemplate = boardTemplate.replacingOccurrences(of: "9", with: "\(TicTacToe.character(board[2][1]))")
        print(boardTemplate, terminator: "\n\n")
    }
}

Now, all that's left is the AI. Minimax can be implemented as a very simple recursive function. This is the function signature:

`minimax(state: TicTacToe, player: Int, depth: Int = 0) -> Double`

This function will take the state that is to be scored, the player that needs to play imminently, and the current depth in the tree of the node that it's processing. Initially, this is, of course, zero.

The function has very simple logic:

1. Does this state represent a "game over"? If so, return the result from `state.winner(depth: Int)`, passing it the depth passed into the `minimax` function. The return type is a double-precision floating-point number.
1. For each child of this state, pass it back into the `minimax` function assuming the next player needs to play and with an incremented depth value.
1. If the current player is 1, return the maximum value of all the children's scores that were returned from `minimax`.
1. If the current player is -1, return the minimum value of all the children's scores that were returned from `minimax`.

In [ ]:
func minimax(state: TicTacToe, player: Int, depth: Int = 0) -> Double {
    if state.isOver { // If the game is over ...
        return Double(state.winner(depth: depth)) // ... return the score.
    }
    let children = state.children(player: player) // Determine the children of the game state ...
                        .map({ minimax(state: $0, player: -player, depth: depth + 1) }) // ... and find their minimax scores.
    if player == 1 { // If the player is 1 (maximizer) ...
        return children.max()! // ... return the score of the child with the highest score.
    } else { // If the player is -1 (minimizer) ...
        return children.min()! // ... return the score of the child with the lowest score.
    }
}

Now that you have a mechanism to return the score for any board, you need to have a way to make a decision as to where to move for any given board state. To do this, simply iterate along every child of a given board state, get the score from Minimax and multiply it by the player, and return the move with the largest score.

But wait - why do we need to multiply the score from minimax by the player? It's for a simple reason - if player 1 has to play, minimax will return a positive value for moves that are in its favor. If player -1 has to player, minimax will return a negative value for moves that are in its favor. However, to make sure that the maximum score corresponds to the best move for the player, simply multiply the score by the player (either `1` or `-1`).

In [ ]:
func decision(for state: TicTacToe, player: Int) -> (Int, Int) {
    let children = state.children(player: player) // Find the children of a game state.
    let scores = children.map({ Double(player) * minimax(state: $0, player: -player) }) // Find their minimax scores and canonicalize the score by multiplying by the player.
    return state.legalMoves[scores.firstIndex(where: { $0 == scores.max()! })!] // Return the move that has the highest minimax score.
}

And... you're good to go. You've implemented a brute-force, tree-based solution to solving the game of Tic Tac Toe. Good job! Now, let's have it play against itself:

In [ ]:
var state = TicTacToe()
var player = 1
state.display()
while !state.isOver {
    state.play(at: decision(for: state, player: player), player: player)
    player = -player
    state.display()
}

Remember, this is a perfect & deterministic player. It'll always play the same move for a certain game state. Meaning that, if you have it play against itself, it'll always result in the same game!

<hr>
<h2>The last exercise!</h2>
<p>Congratulations, you have completed your first lesson and hands-on lab in Python. However, there is one more thing you need to do. The Data Science community encourages sharing work. The best way to share and showcase your work is to share it on GitHub. By sharing your notebook on GitHub you are not only building your reputation with fellow data scientists, but you can also show it off when applying for a job. Even though this was your first piece of work, it is never too early to start building good habits. So, please read and follow <a href=“https://cognitiveclass.ai/blog/data-scientists-stand-out-by-sharing-your-notebooks/” target=“_blank”>this article</a> to learn how to share your work.
<hr>